In [59]:
import numpy as np
import torchvision
import torch

def roi_align(feature_map, boxes, num_obj, im_width, im_height):

    feature_map = feature_map.to(torch.device('cpu'))

    fmap_height, fmap_width = feature_map.shape[2:]
    boxes[:,:,0]/=im_width
    boxes[:,:,2]/=im_width

    boxes[:,:,1]/=im_height
    boxes[:,:,3]/=im_height

    boxes[:,:,0]*=fmap_width
    boxes[:,:,2]*=fmap_width

    boxes[:,:,1]/=fmap_height
    boxes[:,:,3]/=fmap_height
    
    pooler = torchvision.ops.RoIAlign(output_size=(1, 1), spatial_scale = 1.0, sampling_ratio=1)
    
    
    num_frames = 11
    roi_pools = []

    boxes_list = list(boxes.split(1, dim=1))
    boxes_list = [boxes_list[i].squeeze(1) for i in range(len(boxes_list))]
    
    boxes_list_2 = []
    
    for i, b in enumerate(boxes_list):
        boxes_list_2.append( b[:num_obj] )
    
    pooled_output = pooler(feature_map, boxes_list_2)
    return pooled_output

In [60]:
# Master Feature Generator for VSGNet
import torch
import os

def master_feature_generator_old( full_feature_folder, old_data_dict):

    # Get the feature from the full feature folder
    # Load the resnet 152 feature map
    # remove unnecessary keys
    # perform RoIalign on the resnet152 feature map
    # return the result
    
    yt_id = old_data_dict['metadata']['yt_id']
    frame_no = old_data_dict['metadata']['frame no.']
    suffix = '5'
    
    full_feature_filename = yt_id + '_' + str(frame_no) + '_' + suffix + '.pt'
    vsgnet_feature_file_loc = os.path.join( full_feature_folder, full_feature_filename)
    
    full_feature = torch.load(vsgnet_feature_file_loc)
    
    convo_feature_map = full_feature['i3d_fmap']
    bboxes = full_feature['bboxes']
    
    old_data_dict['bboxes'] = bboxes # as old_data_dict bboxes are corrupt
    
    del full_feature
    
    im_width = old_data_dict['metadata']['frame_width']
    im_height = old_data_dict['metadata']['frame_height']
    
    num_obj = old_data_dict['num_obj']
    
    # perform RoI align
    roialign_output = roi_align( convo_feature_map, bboxes, num_obj, im_width, im_height)
    
    result_tensor = torch.zeros((11, 12, 1024))
    
    for frame in range(11):
        ind0 = frame*num_obj
        ind1 = (frame + 1)*num_obj
        result_tensor[frame, :num_obj] = roialign_output[ind0:ind1,0,0]
        
    keep_keys = ['metadata', 'num_obj', 'lr', 'mr', 'cr', 'object_pairs', 'num_relation',
                 'i3d_feature_map', 'bboxes']
    
    new_dict = {}
    
    for k in keep_keys:
        new_dict[k] = old_data_dict[k]
    
    new_dict['cnn_bbox_feature'] = result_tensor
        
    return new_dict


In [1]:
import os
def master_feature_generator( gpnn_feature_dict, old_data_dict):

    
    yt_id = old_data_dict['metadata']['yt_id']
    frame_no = old_data_dict['metadata']['frame no.']

    yt_id_gpnn = gpnn_feature_dict['metadata']['yt_id']
    frame_no_gpnn = gpnn_feature_dict['metadata']['frame no.']
    
    assert yt_id_gpnn==yt_id and frame_no==frame_no_gpnn, "Some Issue"
    

    gpnn_feature_dict['relative_spatial_feature'] = old_data_dict['relative_spatial_feature']
    return gpnn_feature_dict

In [49]:
# vsgnet_feature_folder = '/workspace/data/data_folder/o2o/gifs_11_features_vsgnet'

# import torch

# temp_old_data_loc = '/workspace/data/data_folder/o2o/gifs_11_features/_1_dgZ4-Ldw_2432_5.pt'
# old_data_dict = torch.load(temp_old_data_loc)

# roi_align_res = master_feature_generator(vsgnet_feature_folder, old_data_dict)

debug torch.Size([12, 4])


In [5]:
split = 'val'

features_file = '/workspace/data/data_folder/o2o/all_features/gifs_11/' + split + '.pkl'
gpnn_feature_file = '/workspace/data/data_folder/o2o/all_features/gpnn/' + split + '.pkl'

import pickle as pkl
features = pkl.load(open(features_file,'rb'))
features_gpnn = pkl.load(open(gpnn_feature_file,'rb'))

new_features = []

from tqdm import tqdm as tqdm

for i in tqdm(range(len(features))):
    old_dict = features[i]
    gpnn_dict = features_gpnn[i]
    
    new_dict = master_feature_generator(gpnn_dict, old_dict)
    new_features.append(new_dict)

new_features_file = open('/workspace/data/data_folder/o2o/all_features/gpnn_with_edge/' + split + '.pkl','wb')

pkl.dump(new_features, new_features_file)
new_features_file.close()

100%|██████████| 206/206 [00:00<00:00, 197582.12it/s]


In [61]:
new_dict = master_feature_generator(vsgnet_feature_folder, features[346])

In [50]:
roi_align_res.keys()

dict_keys(['metadata', 'num_obj', 'lr', 'mr', 'cr', 'object_pairs', 'num_relation', 'i3d_feature_map', 'bboxes', 'cnn_bbox_feature'])

In [ ]:
# For combining all features into one pickle file
from tqdm import tqdm as tqdm

def combine_all_pickles(feature_folder):
    from glob import glob as glob
    files = glob(feature_folder + '/*.pt')
    
    all_features = []
    import torch
    
    for f in tqdm(files):
        data = torch.load(f)
        all_features.append(data)
    return all_features

feature_folder = '/workspace/data/data_folder/o2o/gifs_11_features_vsgnet'
data = combine_all_pickles(feature_folder)

saving_loc = '/workspace/data/data_folder/o2o/all_features/gifs_11_vsgnet/all_features_11_vsgnet.pkl'

import pickle as pickle
f = open(saving_loc, 'wb')
pickle.dump(data, f)
f.close()

In [40]:
len(data)

2050

In [41]:
# For creating train, test and val splits using previously generated data

# train, test, validation split

import pickle

folder_loc = '/workspace/work/CVPR22/ooi_classification/hidden/gcn/data'

train_loc = folder_loc + '/training3.pkl'
test_loc = folder_loc + '/testing3.pkl'
val_loc = folder_loc + '/validation3.pkl'

train_data = pickle.load(open(train_loc,'rb'))
val_data = pickle.load(open(val_loc,'rb'))
test_data = pickle.load(open(test_loc,'rb'))


split_dict = {}

for t in train_data:
    yt_id = t['metadata']['yt_id']
    frame_no = t['metadata']['frame no.']
    temp_key = yt_id + '_' + frame_no
    split_dict[temp_key] = 'train'
    
for t in test_data:
    yt_id = t['metadata']['yt_id']
    frame_no = t['metadata']['frame no.']
    temp_key = yt_id + '_' + frame_no
    split_dict[temp_key] = 'test'
    
for t in val_data:
    yt_id = t['metadata']['yt_id']
    frame_no = t['metadata']['frame no.']
    temp_key = yt_id + '_' + frame_no
    split_dict[temp_key] = 'val'
    
saving_path = '/workspace/data/data_folder/o2o/split_dict.pkl'

pickle.dump(split_dict, open(saving_path,'wb'))

In [33]:
# For splitting combined features file into 3 pickle files using split dictionary
import pickle as pkl

def split(split_dict_path, combined_features_file):


    all_features = pkl.load(open(combined_features_file, 'rb'))
    split_dict = pkl.load(open(split_dict_path,'rb'))
    
    train = []
    test = []
    val = []
    
    for feat in all_features:
        yt_id = feat['metadata']['yt_id']
        frame_no = feat['metadata']['frame no.']
        temp_key = yt_id + '_' + frame_no
        split = split_dict[temp_key]
        
        if split == 'train':
            train.append(feat)
        elif split == 'test':
            test.append(feat)
        elif split == 'val':
            val.append(feat)

            

    saving_folder = '/'.join( combined_features_file.split('/')[:-1] )
    import os
    
    train_file = os.path.join(saving_folder, 'train.pkl')
    f = open(train_file, 'wb')
    pkl.dump(train, f)
    f.close()
    
    test_file = os.path.join(saving_folder, 'test.pkl')
    f = open(test_file, 'wb')
    pkl.dump(test, f)
    f.close()

    val_file = os.path.join(saving_folder, 'val.pkl')
    f = open(val_file, 'wb')
    pkl.dump(val, f)
    f.close()
    
    return train, test, val

split_dict_path = '/workspace/data/data_folder/o2o/split_dict.pkl'
combined_features_file = '/workspace/data/data_folder/o2o/all_features/gifs_11/all_features_11.pkl'

train, test, val = split(split_dict_path, combined_features_file)

In [2]:
import torch

test_dir = '/workspace/data/data_folder/o2o/gifs_11_features_vsgnet'
from glob import glob as glob

list_files = glob(test_dir+ '/*.pt')
issue_files = []

from tqdm import tqdm as tqdm
for f in tqdm(list_files):
    
    data = torch.load(f)
    if not torch.is_tensor(data['frame_deep_features']):
        issue_files.append(f)
    del data
print(len(issue_files))
    

100%|██████████| 2055/2055 [09:08<00:00,  3.75it/s] 

4


In [3]:
issue_files

['/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/0dqx7VOjiJI_2426_5.pt',
 '/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/0dqx7VOjiJI_1572_5.pt',
 '/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/0dqx7VOjiJI_2358_5.pt',
 '/workspace/data/data_folder/o2o/gifs_11_features_vsgnet/0dqx7VOjiJI_2357_5.pt']

In [1]:
import torch
o2o_dir = '/workspace/data/data_folder/o2o/gifs_11_features_vsgnet'
from glob import glob as glob

o2o_files = glob(o2o_dir + '/*.pt')



In [14]:
import torch
issues = []

new_keys = ['metadata', 'num_obj', 'bboxes', 'lr', 'mr', 'cr', 'object_pairs', 'num_relation', 'frame_deep_features']

for f in o2o_files:
    
    data = torch.load(f)
    new_data = {}
    
    for n in new_keys:
        new_data[n] = data[n]
    torch.save(new_data, f)
